In [0]:
from requests import get
from bs4 import BeautifulSoup

In [0]:
url = 'http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1'

In [0]:
response = get(url)

In [4]:
print(response.text[:500])




<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle"


In [0]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [0]:
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')

In [7]:
print(len(movie_containers))

50


1. The name of the movie.
2. The year of release.
3. The IMDB rating.
4. The Metascore.
5. The number of votes.

In [0]:
first_movie = movie_containers[0]

In [9]:
first_movie.div

<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt3315342"></div>
</div>

In [10]:
first_movie.a

<a href="/title/tt3315342/"> <img alt="Logan" class="loadlate" data-tconst="tt3315342" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYzc5MTU4N2EtYTkyMi00NjdhLTg3NWEtMTY4OTEyMzJhZTAzXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB466725069_.png" width="67"/>
</a>

In [11]:
first_movie.h3

<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt3315342/">Logan</a>
<span class="lister-item-year text-muted unbold">(2017)</span>
</h3>

In [12]:
first_movie.h3.a.get_text()

'Logan'

Year of the movie release

In [13]:
first_year = first_movie.h3.find('span', class_ = 'lister-item-year text-muted unbold').get_text()
first_year

'(2017)'

In [0]:
details = first_movie.p.find('span', class_ = 'genre')
first_details = str(details.text)

In [15]:
print(first_details)


Action, Drama, Sci-Fi            


In [16]:
first_imdb = float(first_movie.strong.get_text())
first_imdb

8.1

In [0]:
mscore = first_movie.find('span', class_ = 'metascore favorable')
first_mscore = int(mscore.text)

In [18]:
print(first_mscore)

77


In [19]:
votes = first_movie.find('span', attrs={'name':'nv'})
print(votes.text)

607,481


In [20]:
votes['data-value']

'607481'

In [0]:
first_votes = int(votes['data-value'])

In [22]:
first_votes

607481

In [0]:
tm = movie_containers[22].find('div',class_ = 'ratings-metascore')

In [24]:
type(tm)

NoneType

In [0]:
Names = []
Years = []
IMDB_ratings = []
MetaScore = []
Votes = []
Gener = []

for Container in movie_containers:
  if Container.find('div', class_ ='ratings-metascore') is not None:
    name = Container.h3.a.text
    Names.append(name)

    year = Container.h3.find('span', class_ = 'lister-item-year text-muted unbold').text
    Years.append(year)

    imdb = float(Container.strong.text)
    IMDB_ratings.append(imdb)

    m_score = Container.find('span', class_ = 'metascore favorable')
    MetaScore.append(m_score)

    votes = Container.find('span', attrs={'name':'nv'})['data-value']
    Votes.append(votes)

    gener = Container.p.find('span', class_ = 'genre').text
    Gener.append(gener)

In [0]:
import pandas as pd
IMDB_Data = pd.DataFrame({
    'Name':Names,
    'Year':Years,
    'Ratings':IMDB_ratings,
    'Score':MetaScore,
    'Votes':Votes,
    'Gener':Gener
})

In [27]:
print(IMDB_Data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Name     42 non-null     object 
 1   Year     42 non-null     object 
 2   Ratings  42 non-null     float64
 3   Score    27 non-null     object 
 4   Votes    42 non-null     object 
 5   Gener    42 non-null     object 
dtypes: float64(1), object(5)
memory usage: 2.1+ KB
None


In [0]:
pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(2000,2018)]

In [0]:
from IPython.core.display import clear_output
from time import time
from time import sleep
from random import randint

In [30]:
Names = []
Years = []
IMDB_ratings = []
MetaScore = []
Votes = []
Gener = []

start_time = time()
requests = 0

for year_url in years_url:
   
   for page in pages:
     response = get('http://www.imdb.com/search/title?release_date=' + year_url +
                    '&sort=num_votes,desc&page=' + page)
     
     sleep(randint(8,15))

     requests+=1
     elapsed_time = time() - start_time
     print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
     clear_output(wait = True)

     if response.status_code != 200:
       warn('Request: {}; Status code: {}'.format(requests, response.status_code))

     if requests > 72:
       warn('Number of requests was greater than expected.')
       break

     page_html = BeautifulSoup(response.text,'html.parser')
     mv_containers = page_html.find_all('div', 'lister-item mode-advanced')

     for Container in movie_containers:
       if Container.find('div', class_ ='ratings-metascore') is not None:
          name = Container.h3.a.text
          Names.append(name)

          year = Container.h3.find('span', class_ = 'lister-item-year text-muted unbold').text
          Years.append(year)

          imdb = float(Container.strong.text)
          IMDB_ratings.append(imdb)

          m_score = Container.find('span', class_ = 'metascore favorable')
          MetaScore.append(m_score)

          votes = Container.find('span', attrs={'name':'nv'})['data-value']
          Votes.append(votes)

          gener = Container.p.find('span', class_ = 'genre').text
          Gener.append(gener)

Request:72; Frequency: 0.08090895733225932 requests/s


In [0]:
import pandas as pd
IMDB_Data = pd.DataFrame({
    'Name':Names,
    'Year':Years,
    'Ratings':IMDB_ratings,
    'Score':MetaScore,
    'Votes':Votes,
    'Gener':Gener
})

In [32]:
print(IMDB_Data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3024 entries, 0 to 3023
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Name     3024 non-null   object 
 1   Year     3024 non-null   object 
 2   Ratings  3024 non-null   float64
 3   Score    1944 non-null   object 
 4   Votes    3024 non-null   object 
 5   Gener    3024 non-null   object 
dtypes: float64(1), object(5)
memory usage: 141.9+ KB
None


In [0]:
#IMDB_Data.to_csv('IMDB_DATA')

In [0]:
#!cp data.csv 'drive/My Drive/'